✅ Пайплайн обработки PhotoMaps

📂 Исходные данные:
	•	Исходные фото → images/*.jpg
    
    
🔧 Шаг 1 — Сжатие фото
	•	Откуда: images/
	•	Куда: photos/
	•	Что делает: уменьшает до 1600x1600, сохраняет EXIF, перезаписывает JPG с качеством ~85%
	•	✅ Используется: Image.thumbnail() + ImageOps.exif_transpose()
    
📋 Шаг 2 — Генерация CSV
	•	Источник: photos/*.jpg
	•	Результат: csv/photos.csv
	•	Что делает: парсит EXIF, извлекает координаты, дату, имя файла
    
🎨 Шаг 3 — Генерация превью
	•	Источник: photos/
	•	Результат: thumbnails/*.png
	•	Что делает: круглые PNG 64x64 с цветной рамкой по году
    
Перед стартом этого пайплайна стоит:
	1.	Очистить папки photos/ и thumbnails/:
    
bash:

rm -rf photos/*

rm -rf thumbnails/*

---

2.	Затем запускаешь ячейки в таком порядке:
    
- Сжатие (images → photos)
- EXIF → CSV (photos → csv/photos.csv)
- Превью PNG (photos → thumbnails)

# Step 1. Script compresses raw jpegs into smaller ones

In [74]:
import os
import piexif
import json
import csv
from PIL import Image
from tqdm import tqdm

# === Константы ===
ROOT = os.getcwd()
IMAGES_FOLDER = os.path.join(ROOT, "images")
PHOTOS_FOLDER = os.path.join(ROOT, "photos")
CSV_FOLDER = os.path.join(ROOT, "csv")
os.makedirs(PHOTOS_FOLDER, exist_ok=True)
os.makedirs(CSV_FOLDER, exist_ok=True)
CSV_LOG = os.path.join(CSV_FOLDER, "errors_compression_log.csv")

MAX_SIZE = (1600, 1600)

def dms_rational(deg_float):
    d = int(deg_float)
    m_float = (deg_float - d) * 60
    m = int(m_float)
    s = int((m_float - m) * 60 * 100)
    return [(d, 1), (m, 1), (s, 100)]

def create_gps_ifd(lat, lon):
    return {
        piexif.GPSIFD.GPSLatitudeRef: b'N' if lat >= 0 else b'S',
        piexif.GPSIFD.GPSLatitude: dms_rational(abs(lat)),
        piexif.GPSIFD.GPSLongitudeRef: b'E' if lon >= 0 else b'W',
        piexif.GPSIFD.GPSLongitude: dms_rational(abs(lon))
    }

def extract_coords_from_exif(img):
    try:
        exif_data = piexif.load(img.info.get("exif", b""))
        gps = exif_data.get("GPS", {})
        if piexif.GPSIFD.GPSLatitude in gps and piexif.GPSIFD.GPSLongitude in gps:
            lat_vals = gps[piexif.GPSIFD.GPSLatitude]
            lon_vals = gps[piexif.GPSIFD.GPSLongitude]
            lat = lat_vals[0][0]/lat_vals[0][1] + lat_vals[1][0]/lat_vals[1][1]/60 + lat_vals[2][0]/lat_vals[2][1]/3600
            lon = lon_vals[0][0]/lon_vals[0][1] + lon_vals[1][0]/lon_vals[1][1]/60 + lon_vals[2][0]/lon_vals[2][1]/3600
            if gps.get(piexif.GPSIFD.GPSLatitudeRef) == b'S': lat = -lat
            if gps.get(piexif.GPSIFD.GPSLongitudeRef) == b'W': lon = -lon
            return lat, lon
    except: return None, None
    return None, None

def extract_coords_from_json(json_path):
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            geo = data.get("geoData", {})
            lat = geo.get("latitude")
            lon = geo.get("longitude")
            return float(lat), float(lon) if lat and lon else (None, None)
    except: return None, None

# === Основной цикл ===
errors = []
images = [f for f in os.listdir(IMAGES_FOLDER) if f.lower().endswith((".jpg", ".jpeg"))]

for fname in tqdm(images, desc="📦 Сжатие изображений", ncols=80):
    src = os.path.join(IMAGES_FOLDER, fname)
    dst = os.path.join(PHOTOS_FOLDER, fname)
    try:
        img = Image.open(src)
        lat, lon = extract_coords_from_exif(img)
        if lat is None or lon is None:
            lat, lon = extract_coords_from_json(src + ".supplemental-me.json")
        if lat is None or lon is None:
            errors.append([fname, "no_coordinates"])
            continue
        img.thumbnail(MAX_SIZE, Image.LANCZOS)
        gps_ifd = create_gps_ifd(lat, lon)
        exif_bytes = piexif.dump({"GPS": gps_ifd})
        img.convert("RGB").save(dst, "jpeg", quality=85, optimize=True, exif=exif_bytes)
    except Exception as e:
        errors.append([fname, str(e)])

# === Сохраняем лог ошибок ===
with open(CSV_LOG, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "error"])
    writer.writerows(errors)

print(f"✅ Обработка завершена. Успешно: {len(images) - len(errors)} | Пропущено: {len(errors)} | Лог: {CSV_LOG}")

📦 Сжатие изображений: 100%|██████████████████| 662/662 [01:58<00:00,  5.61it/s]

✅ Обработка завершена. Успешно: 586 | Пропущено: 76 | Лог: /Users/mloktionov/PycharmProjects/PhotoMaps/csv/errors_compression_log.csv


In [ ]:
# Step 2. Create csv file with photos

# (extract coordinates, year, month, date from photo info and puts this data along with photo ID into csv file

In [76]:
import os
import pandas as pd
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import re
from tqdm import tqdm

# === 📁 Параметры ===
ROOT = os.path.abspath(".")
IMAGE_FOLDER = os.path.join(ROOT, "photos")
CSV_FOLDER = os.path.join(ROOT, "csv")
os.makedirs(CSV_FOLDER, exist_ok=True)

CSV_OUTPUT = os.path.join(CSV_FOLDER, "photos.csv")
ERROR_LOG = os.path.join(CSV_FOLDER, "errors_csv_log.csv")

# === 🧭 EXIF helpers ===
def convert_to_degrees(v):
    d, m, s = v
    return float(d) + float(m)/60 + float(s)/3600

def extract_gps(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if not exif_data:
            return None, None
        gps_raw = {}
        for tag_id, val in exif_data.items():
            tag = TAGS.get(tag_id)
            if tag == "GPSInfo":
                for key in val:
                    gps_tag = GPSTAGS.get(key)
                    gps_raw[gps_tag] = val[key]
        if 'GPSLatitude' in gps_raw and 'GPSLongitude' in gps_raw:
            lat = convert_to_degrees(gps_raw['GPSLatitude'])
            if gps_raw.get('GPSLatitudeRef', 'N') != 'N':
                lat = -lat
            lon = convert_to_degrees(gps_raw['GPSLongitude'])
            if gps_raw.get('GPSLongitudeRef', 'E') != 'E':
                lon = -lon
            return lat, lon
    except Exception as e:
        print(f"❌ Ошибка при обработке {image_path}: {e}")
    return None, None

def extract_date_from_filename(filename):
    match = re.search(r'IMG_(\d{4})(\d{2})(\d{2})', filename)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3))
    return None, None, None

# === 📊 Обработка изображений ===
records = []
errors = []

images = [f for f in os.listdir(IMAGE_FOLDER) if f.lower().endswith(".jpg")]

for fname in tqdm(images, desc="📷 Генерация CSV", ncols=80):
    full_path = os.path.join(IMAGE_FOLDER, fname)
    lat, lon = extract_gps(full_path)
    year, month, day = extract_date_from_filename(fname)

    if lat is None or lon is None:
        errors.append({'filename': fname})
        continue

    records.append({
        'filename': fname,
        'folder': os.path.basename(IMAGE_FOLDER),
        'latitude': lat,
        'longitude': lon,
        'year': year,
        'month': month,
        'day': day
    })

# === 💾 Сохраняем CSV
df = pd.DataFrame(records)
df.to_csv(CSV_OUTPUT, index=False)
print(f"✅ Сохранено: {CSV_OUTPUT} ({len(df)} записей)")

# === ⚠️ Сохраняем ошибки (если есть)
if errors:
    pd.DataFrame(errors).to_csv(ERROR_LOG, index=False)
    print(f"⚠️ Пропущено {len(errors)} изображений без координат. Список: {ERROR_LOG}")
else:
    print("✅ Все изображения содержали координаты.")

📷 Генерация CSV: 100%|█████████████████████| 586/586 [00:00<00:00, 3798.43it/s]

✅ Сохранено: /Users/mloktionov/PycharmProjects/PhotoMaps/csv/photos.csv (586 записей)
✅ Все изображения содержали координаты.


##### 

In [ ]:
# ------ Generating preview thumbnails in circles of a certain color

In [84]:
import os
import pandas as pd
from PIL import Image, ImageDraw
from tqdm import tqdm

# === Параметры ===
ROOT = os.getcwd()
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
INPUT_FOLDER = os.path.join(ROOT, "photos")
THUMBNAIL_FOLDER = os.path.join(ROOT, "thumbnails")
SIZE = (64, 64)
BORDER_WIDTH = 4
FORMAT = "PNG"

# === Цвета по годам ===
palette = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b",
    "#e377c2", "#7f7f7f", "#bcbd22", "#17becf", "#aec7e8", "#ffbb78",
    "#98df8a", "#ff9896", "#c5b0d5", "#c49c94", "#f7b6d2", "#c7c7c7",
    "#dbdb8d", "#9edae5", "#393b79", "#637939", "#8c6d31", "#843c39",
    "#7b4173", "#3182bd", "#e6550d", "#31a354", "#756bb1", "#636363",
    "#17becf", "#bcbd22"
]
YEAR_COLORS = {
    year: palette[i % len(palette)] for i, year in enumerate(range(2000, 2032))
}
YEAR_COLORS["default"] = "#999999"

# === Убедимся, что папка есть
os.makedirs(THUMBNAIL_FOLDER, exist_ok=True)

# === Загрузка CSV
df = pd.read_csv(CSV_PATH)

# === Генерация круглой иконки
def create_thumbnail(image_path, output_path, border_color):
    try:
        img = Image.open(image_path).convert("RGBA")
        img = img.resize(SIZE, Image.LANCZOS)

        # Маска круга
        mask = Image.new('L', SIZE, 0)
        draw = ImageDraw.Draw(mask)
        draw.ellipse((0, 0, SIZE[0], SIZE[1]), fill=255)

        # Применяем маску
        result = Image.new('RGBA', SIZE)
        result.paste(img, (0, 0), mask)

        # Рамка
        draw = ImageDraw.Draw(result)
        inset = BORDER_WIDTH // 2
        draw.ellipse(
            (inset, inset, SIZE[0] - inset - 1, SIZE[1] - inset - 1),
            outline=border_color, width=BORDER_WIDTH
        )

        result.save(output_path, FORMAT)

    except Exception as e:
        print(f"❌ Ошибка для {image_path}: {e}")

# === Основной цикл
for _, row in tqdm(df.iterrows(), total=len(df), desc="🌀 Генерация PNG", ncols=80):
    fname = row["filename"]
    year = row["year"]
    color = YEAR_COLORS.get(year, YEAR_COLORS["default"])
    
    in_path = os.path.join(INPUT_FOLDER, fname)
    out_name = fname.replace(".jpg", ".png").replace(".jpeg", ".png").replace(".JPG", ".png")
    out_path = os.path.join(THUMBNAIL_FOLDER, out_name)

    create_thumbnail(in_path, out_path, color)

print(f"✅ Превьюшки сохранены в: {THUMBNAIL_FOLDER}")

🌀 Генерация PNG: 100%|███████████████████████| 586/586 [00:16<00:00, 36.60it/s]

✅ Превьюшки сохранены в: /Users/mloktionov/PycharmProjects/PhotoMaps/thumbnails
